<a href="https://colab.research.google.com/github/mastdev/pos_tag/blob/main/hmm_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
from collections import defaultdict
import os
import xml.etree.ElementTree as et
import pandas as pd


Pre Processing


In [90]:
path='/content/drive/My Drive/Ai_project_hmm/Train-corpus'
transition_matrix = defaultdict(dict)
probability_matrix = defaultdict(dict)
initial_matrix = defaultdict(dict)
different_tags = []
different_words = {}

In [ ]:
co_trans = defaultdict(dict)
co_tag_word = defaultdict(dict)
co_initial = defaultdict(dict)
p=[.75,.25]

for _dir,subdir,files in os.walk(path):
    for file in files:
            file_path=os.path.join(_dir,file)
            tree=et.parse(file_path)
            root=tree.getroot()
            prev = []
            prev_word='.'
            for word in root.findall('.//'):
                if word.tag=='w' or word.tag=='c' and w!=None:
                    w=word.text
                    tag=word.attrib.get('c5')
                    w=w.lower()
                    w=w.replace(' ','')
                    if w not in different_words:
                        different_words[w] = 1
                    taglist = tag.split('-')
                    if len(taglist)==1 :
                        taglist.append(taglist[0])

                    for i in taglist:
                        if i not in different_tags:
                            different_tags.append(i)
                    
                    co = 0;

                    for j in taglist:
                        if w not in co_tag_word[j] :
                            co_tag_word[j][w]=0
                        co_tag_word[j][w] += p[co]
                        if prev_word == '.':
                            if co_initial[j]=={}:
                                co_initial[j]=0
                            co_initial[j]+=p[co]
                        co+=1
                    if prev!=[]:
                        for i in range(0,2):
                            for j in range(0,2):
                                if taglist[j] not in co_trans[prev[i]]: 
                                    co_trans[prev[i]][taglist[j]]=0
                                co_trans[prev[i]][taglist[j]]+=p[i]*p[j]
                    prev = taglist


new_word = "#un#"
load = {}
for i in different_tags:
    x = .5
    if i=='NP0' :
        x=.8
    load[i] = x;

for tag in co_tag_word:
    min_key = min(co_tag_word[tag],key = co_tag_word[tag].get)
    co_tag_word[tag][new_word]=load[tag]*co_tag_word[tag][min_key]

co_initial_sum = sum(co_initial.values())
for tag in co_initial:
    initial_matrix[tag] = co_initial[tag]/co_initial_sum
co_trans_sum = 0;
for x in co_trans:
    co_trans_sum += sum(co_trans[x].values())
for i in co_trans:
    for j in co_trans[i]:
        if j not in transition_matrix[i]:
             transition_matrix[i][j]=0
        transition_matrix[i][j] += co_trans[i][j]/co_trans_sum
for tag in co_tag_word:
    local_sum = sum(co_tag_word[tag].values())
    for x in co_tag_word[tag]:
        if x not in probability_matrix[tag]:
             probability_matrix[tag][x]=0
        probability_matrix[tag][x] += co_tag_word[tag][x]/local_sum

for tag in different_tags:
    if tag not in initial_matrix:
        initial_matrix[tag]=0
    for subtag in different_tags:
        if subtag not in transition_matrix[tag]:
            transition_matrix[tag][subtag]=0

df1 = pd.DataFrame.from_dict(transition_matrix)
df2 = pd.DataFrame.from_dict(probability_matrix)
df3 = pd.DataFrame(initial_matrix, index =[0])

df1.to_csv('/content/drive/My Drive/Ai_project_hmm/transition_matrix')
df2.to_csv('/content/drive/My Drive/Ai_project_hmm/probability_matrix')
df3.to_csv('/content/drive/My Drive/Ai_project_hmm/initial_matrix')


